## EE-361M Introduction to Data Mining
## Assignment #2
## Due: Thursday, Feb 18, 2016 by 2pm; Total points: 50


Your homework should be written in a **Jupyter notebook** (if this isn't possible, let me know). Please use this naming format for your notebook you submit: **Group(Group Num)_HW(HW Number).ipynb**. For example, Group1_HW1.ipynb. Homeworks are due at the beginning of class on the due date and should be submitted through Canvas in your **groups of 3 from the first homework**. If groups need to be adjusted please contact the TA.

## Question 1: Sampling
### 10 points

1. CBS has come up with an extreme TV show, and each of its viewers either likes or hates it. (no middle ground here; we are in a 'black and white age'). CBS wants to estimate what fraction $p$ of its audience like the show by 'randomly' calling $n$ viewers and tallying their responses so as to estimate the true value of $p$ to a fractional  accuracy of within $\pm \epsilon$%, with a confidence of $(1-\alpha) \times 100$%. For $\alpha =  0.1$, $\epsilon = 0.02$ (i.e. your answer will be $\hat{p} \pm 0.02$), what is the minimum value of $n$ needed if (i) true value $p = 0.5$ and (ii) $p = 0.95$? 
%(First try to do this yourself knowing that you have a binomial distribution, which can be approximated by a normal distribution. If you cannot, consult an undergrad stats book.)
2. Suppose for a certain value  of $p$ and choice of $\epsilon$, you calculate that you will need (at least) 1000 samples for $\alpha = 0.1$. You now decide to obtain  a more accurate answer by either (i) reducing $\alpha$ to 0.05, keeping the same $\epsilon$ or by (ii) reducing $\epsilon$ by a factor of 2 from the original value, but maintaining  $\alpha = 0.1$.  In each case how many samples would you need now?

## Question 2: Republican Presidental Debate
### 10 points

In this question we will be analyzing text data from one of the recent presidental debates. I have included code below to grab the data for you from the New York Times.

1. Create a set of the frequency of utterance of  all the distinct words spoken by candidates, and then use it to create a histogram (with 30 bins) of word counts. Thus a bin is a range of count values and the corresponding "y" value is the number of words whose count falls in this range. What is interesting about this distribution? What are the 10 most common words?
2. Remove the 100 most common words from vocabulary. Meaning that if you ever see this word, get rid of it. Now create a new python dictionary for each candidate that is a single list of all the words spoken by this candidate (ignoring these most common words). What are the 10 most common words for Trump, Rubio, and Cruz? How do their words differ?
3. Using our dictionary from number 2, how many words did each speaker speak? Who spoke the most? Who is the outlier?
4. Count the percentage of time each person uses the words (I, I'm, me, mine). When doing this convert all words to lower case. Create a bar plot of this percentage for each candidate with bars from largest to smallest. Use dictionary that has all words (doesn't exclude most common). What does the plot show?

Hints:
1. Look at python Counter.
2. Just split text on a space. This isn't perfect, but will be fine.

In [1]:
import requests
from bs4 import BeautifulSoup
from collections import defaultdict
from collections import Counter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

url = 'http://www.nytimes.com/2015/11/11/us/politics/transcript-republican-presidential-debate.html'
# requests gets the source code from the url and extracts it as text
html = requests.get(url).text
# beautifulsoup is a library that takes in text source code and returns a structured format of that
# source code that you can more easily search and parse.
soup = BeautifulSoup(html, 'html5lib')
# get all the 'p' tags from the source with class = 'story-body-text'
# this was determined by looking at the source code
# the first and last paragraphs are intro and ending
paragraphs = soup('p', {'class': 'story-body-text'})[1:-1]
candidates = ['BUSH', 'TRUMP', 'RUBIO', 
              'CARSON', 'FIORINA', 'KASICH', 'CRUZ', 'PAUL']

def text_to_dict(paragraph_array):
    '''takes an array of text paragraphs from debate and returns dict 
    where key is person and value is list of text spoken by that candidate'''
    # dict is like a hash map. defaultdict lets you specify what types of values will be in your hash map
    d = defaultdict(list)
    # just a default speaker that won't end up in our returned data
    # will get replaced when an actual speaker is found
    speaker = "<START>"
    for paragraph in paragraph_array:
        words = paragraph.text.split(' ')
        first_word = words[0]
        # only new speaker when have SPEAKER: format
        if first_word[-1] == ":":
            speaker = first_word[:-1]
        # only keep candidates text
        if speaker in candidates:
            d[speaker].append(words[1:])
    return d

speaker_dict = text_to_dict(paragraphs)

In [31]:
#part 1
word_array = []

for person in speaker_dict:
    for paragraph in speaker_dict[person]:
        for word in paragraph:
            word_array.append(word)
c = Counter(word_array)

s = pd.Series(c)
s.plot(kind="bar")

print "10 Most common words are:"
for letter, count in c.most_common(10):
    print '%s:\t%d' % (letter, count)

10 Most common words are:
the:	775
to:	508
a:	365
of:	360
and:	352
we:	267
that:	261
is:	258
in:	254
I:	232


## Queston 3: Principal Component Analysis
### 15 points

In this question, you will explore an application of PCA.

1. Convert your data from 3.2 to a vectorized format. This means you will have a row for each candidate and a column for each word in your data. A column for a candidate will contain the number of times that candidate used that word. Use [CountVectorizer](http://scikit-learn.org/stable/modules/feature_extraction.html) from sklearn with min_df = 1.
2. Convert your data from a sparse matrix to a dense array using .toarray() and then scale it to have mean zero and standard deviation of 1. See [here](http://scikit-learn.org/stable/modules/preprocessing.html) for help.
2. Plot the explained variance as a function of the number of PCA components (called a scree plot). Use sklearn's PCA functionality to do this.
3. Now pick the top two principal components and project the data onto the respective dimensions. Visualize the data in a scatter plot and label each point with the candidate's name. Who are the outliers? Use sklearn and matplotlib for this. 
4.  In what sense is PCA an optimal feature extraction technique? Describe a situation where you would prefer feature selection to (linear) feature extraction, even though the former  is a special case of the latter.

## Question 4: Robust Regression
### 5 points

In this question we will be exploring using a regression technique that is more robust to outliers. I provide some code below that injects outlier points into the original medv and lstat data from the housing dataset. This problem looks into how robust regression can help in the presence of outliers.

1. Using the original data, plot lstat on the x-axis and log(medv) on the y-axis of a scatter plot with the line of best fit from a linear regression on the plot as well. Do the same, but with the data that includes the outlier values. What changes with the best fit line? Specifically, how does the slope change?
2. Now run a linear regression with a Huber loss on the data including the outliers and create the same plot as above, but this time with the fit from the Huber loss regression (using all the data). What has changed (comment on the slope as well)? Note: Use SGDRegressor from sklearn with 500 iterations and no penalty.
3. Explain why the huber loss is more robust to outliers.

Note:  Use plot's with xlim = (-5, 40) and ylim = (1, 5). These set the range for the x any y axes.

In [3]:
import pandas as pd
import numpy as np

housing_data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/housing/housing.data",
                   delim_whitespace=True, header=None,
                   names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO',
                           'B', 'LSTAT', 'MEDV'])
housing_data = housing_data.dropna()
lstat = housing_data.LSTAT.values
medv = housing_data.MEDV.values
medv_std = np.std(medv)
lstat_std = np.std(lstat)
np.random.seed(42)
medv_outliers = np.random.normal(1, 1, 5)
lstat_outliers = np.random.normal(1, 1, 5)
medv_with_outliers = np.append(medv, medv_outliers)
lstat_with_outliers = np.append(lstat, lstat_outliers)

# Question 5: Visualization using Bokeh
## 10 points

In this problem, you'll build an interactive visualization. Bokeh is a Python interactive visualization library that targets modern web browsers for presentation. For more information on Bokeh, see http://bokeh.pydata.org/en/latest/. The problem statement is as follows:

Using the [auto-mpg](http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data-original) data, your goal is to build a Bokeh visualization which allows the user explore how MPG varies with horsepower and weight. You will create a visualization that allows the user to toggle the Y axis of a scatter plot between horsepower and weight. With the x-axis always being MPG.

Hints: 
1. You can make use of Select widgets.
2. See: http://bokeh.pydata.org/en/latest/docs/user_guide/interaction.html#javascript-callbacks. Specifically look at the CustomJS for Widgets under Callbacks and the Select widget. 
3. See: http://bokeh.pydata.org/en/latest/docs/reference/plotting.html. Look for the scatter API.
4. See: http://bokeh.pydata.org/en/0.10.0/docs/user_guide/styling.html#labels. For labeling axes.
5. Use output_notebook() from Bokeh to output the plot to your notebook

We have made available a sample screenshot of our Bokeh app that supports the above requirements. Your interface should look similar to the screenshots.

In [37]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import show, output_notebook
from bokeh.plotting import figure
import pandas as pd

data = pd.read_table("auto-mpg.data-original.txt")
data.head()

#output_notebook()

#p1 = figure(plot_width=300, plot_height=300)
#p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="navy", alpha=0.5)
#tab1 = Panel(child=p1, title="circle")

#p2 = figure(plot_width=300, plot_height=300)
#p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="navy", alpha=0.5)
#tab2 = Panel(child=p2, title="line")

#tabs = Tabs(tabs=[ tab1, tab2 ])

#show(tabs)




,18.0 8. 307.0 130.0 3504. 12.0 70. 1.,chevrolet chevelle malibu
0,15.0 8. 350.0 165.0 3693. 1...,buick skylark 320
1,18.0 8. 318.0 150.0 3436. 1...,plymouth satellite
2,16.0 8. 304.0 150.0 3433. 1...,amc rebel sst
3,17.0 8. 302.0 140.0 3449. 1...,ford torino
4,15.0 8. 429.0 198.0 4341. 1...,ford galaxie 500
